---
# **Dependencies**
---

In [ ]:
#한글 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# h2o 설치
! apt-get install default-jre
!java -version
! pip install h2o

#런타임 다시 시작
import os
os.kill(os.getpid(), 9)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (3,154 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

---
# **Import Packages**
---

In [1]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')
mount_dir = '/content/drive/MyDrive/weather_con'

sns.set_style("white")
sns.set_context("talk")
plt.rc('font', family='NanumBarunGothic')

import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

import h2o
h2o.init()
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid import H2OGridSearch
from h2o.automl import H2OAutoML
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

Mounted at /content/drive
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvjv4j6yr
  JVM stdout: /tmp/tmpvjv4j6yr/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpvjv4j6yr/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_unknownUser_yawelp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


---
# **Import Data**
---

In [2]:
train_dt = pd.read_csv(mount_dir+'/train_data.csv', encoding='cp949', index_col=0)
train_dt['yyyymmdd'] = pd.to_datetime(train_dt['yyyymmdd'], format = '%Y-%m-%d')

test_dt = pd.read_csv(mount_dir+'/test_data.csv', encoding='cp949', index_col=0)
test_dt['yyyymmdd'] = pd.to_datetime(test_dt['yyyymmdd'], format = '%Y-%m-%d')
test_dt.drop(columns='frequency', inplace=True)

train = train_dt[train_dt['yyyymmdd'] < datetime(2015,7,1)]
valid = train_dt[train_dt['yyyymmdd'] >= datetime(2015,7,1)]

hf_train = h2o.H2OFrame(train)
hf_valid = h2o.H2OFrame(valid)
hf_test = h2o.H2OFrame(test_dt)

print(train_dt.shape, test_dt.shape)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
(49674, 68) (12444, 64)


In [3]:
# 분석 변수 분류
main_col2 = ['area', 'sex', 'weekday', 'is_firstday', 'is_holiday']
fore_col2 = ['예보최대기온', '예보최소기온', '예보평균기온', '최소예보습도', '평균예보습도', '예보풍속']
lag_col2 = ['1일 전 평균기온', '1일 전 최저기온', '1일 전 최고기온', '1일 전 평균풍속', '1일 전 평균이슬점온도', '1일 전 최소상대습도', '1일 전 평균상대습도', '1일 전 최고해면기압',
       '1일 전 최저해면기압', '1일 전 합계일조시간', '1일 전 일교차', '1일 전 기압차', '1일 전 SO2',  '1일 전 CO', '1일 전 O3', '1일 전 NO2', '1일 전 PM10']
acc_col2 = ['acc_causalty_1000pop']
pop_col2 = [ 'pop', 'old']
health_col2 = ['hpts_rate', 'dbts_rate', 'old_rate' ]
hospital_col2 = ['n_doctor_per1000pop', 'n_bed_per1000pop']

predictors = main_col2 + fore_col2 + lag_col2 + acc_col2 + pop_col2 + hospital_col2 + health_col2 + ['log_sum_rn']
response_col = 'frequency'
print(len(predictors))

37


---
# **Modeling**
---

## best DRF, GBM

### 🟩 DRF grid

In [133]:
drf_sc = {"strategy": "Cartesian" }
drf_hp =  { 'mtries' : [24],"max_depth": [9] ,"min_rows" : [17, 19], 'ntrees' : [100], 'sample_rate' : [0.45, 0.55]}
drf_model= H2ORandomForestEstimator(nfolds=5, fold_assignment="Modulo", seed=1234, keep_cross_validation_predictions=True, score_each_iteration = True)
drf_grid= H2OGridSearch(model=drf_model, hyper_params=drf_hp, search_criteria=drf_sc, grid_id="drf_grid")
%time drf_grid.train(x = predictors, y = response_col, training_frame=hf_train, validation_frame=hf_valid)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
CPU times: user 7.49 s, sys: 820 ms, total: 8.31 s
Wall time: 12min 42s
     max_depth  min_rows  mtries  ntrees  sample_rate         model_ids  \
0          9.0      17.0    24.0   100.0         0.55  drf_grid_model_3   
1          9.0      19.0    24.0   100.0         0.55  drf_grid_model_4   
2          9.0      17.0    24.0   100.0         0.45  drf_grid_model_1   
3          9.0      19.0    24.0   100.0         0.45  drf_grid_model_2   

   residual_deviance  
0           1.468148  
1           1.468430  
2           1.470105  
3           1.470998  


In [134]:
drf_rmse = drf_grid.sorted_metric_table()
drf_rmse['xval_rmse'] = np.sqrt(drf_rmse['residual_deviance'] )
drf_rmse['valid_rmse'] = pd.Series([drf_grid.models[i].rmse(valid=True) for i in list(np.arange(0,len(drf_rmse),1))])
drf_rmse

,,max_depth,min_rows,mtries,ntrees,sample_rate,model_ids,residual_deviance,xval_rmse,valid_rmse
0,,9.0,17.0,24.0,100.0,0.55,drf_grid_model_3,1.468148,1.211671,1.223613
1,,9.0,19.0,24.0,100.0,0.55,drf_grid_model_4,1.468430,1.211788,1.223300
2,,9.0,17.0,24.0,100.0,0.45,drf_grid_model_1,1.470105,1.212479,1.223371
3,,9.0,19.0,24.0,100.0,0.45,drf_grid_model_2,1.470998,1.212847,1.222913


### 🟩 GBM grid

In [135]:
gbm_sc = {"strategy": "Cartesian" }
gbm_hp= { 'learn_rate':  [0.085, 0.089, 0.096, 0.098],'ntrees': [100, 130, 140],'learn_rate_annealing' : [0.99],
         'max_depth' : [3, 4],'min_rows' : [130, 200],'sample_rate' : [0.7], 'col_sample_rate_per_tree' :  [0.9]}
gbm_model= H2OGradientBoostingEstimator(nfolds=5, fold_assignment="Modulo", seed=1234, keep_cross_validation_predictions=True, score_each_iteration = True)
gbm_grid= H2OGridSearch(model=gbm_model, hyper_params=gbm_hp, search_criteria=gbm_sc, grid_id="gbm_grid")
%time gbm_grid.train(x = predictors, y = response_col, training_frame=hf_train, validation_frame=hf_valid)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
CPU times: user 44.1 s, sys: 4.51 s, total: 48.6 s
Wall time: 1h 15min 35s
      col_sample_rate_per_tree  learn_rate  learn_rate_annealing  max_depth  \
0                          0.9       0.096                  0.99        3.0   
1                          0.9       0.096                  0.99        3.0   
2                          0.9       0.098                  0.99        3.0   
3                          0.9       0.098                  0.99        3.0   
4                          0.9       0.089                  0.99        3.0   
5                          0.9       0.089                  0.99        3.0   
6                          0.9       0.096                  0.99        3.0   
7                          0.9       0.096                  0.99        3.0   
8                          0.9       0.096                  0.99        3.0   
9                          0.9       0.0

In [136]:
gbm_rmse = gbm_grid.sorted_metric_table()
gbm_rmse['xval_rmse'] = np.sqrt(gbm_rmse['residual_deviance'] )
gbm_rmse['valid_rmse'] = pd.Series([gbm_grid.models[i].rmse(valid=True) for i in list(np.arange(0,len(gbm_rmse),1))])
gbm_rmse

,,col_sample_rate_per_tree,learn_rate,learn_rate_annealing,max_depth,min_rows,ntrees,sample_rate,model_ids,residual_deviance,xval_rmse,valid_rmse
0,,0.9,0.096,0.99,3.0,130.0,140.0,0.7,gbm_grid_model_35,1.454980,1.206225,1.220013
1,,0.9,0.096,0.99,3.0,130.0,130.0,0.7,gbm_grid_model_19,1.455111,1.206280,1.220163
2,,0.9,0.098,0.99,3.0,130.0,140.0,0.7,gbm_grid_model_36,1.455439,1.206416,1.219644
3,,0.9,0.098,0.99,3.0,130.0,130.0,0.7,gbm_grid_model_20,1.455480,1.206433,1.219719
4,,0.9,0.089,0.99,3.0,130.0,140.0,0.7,gbm_grid_model_34,1.455750,1.206545,1.219897
5,,0.9,0.089,0.99,3.0,130.0,130.0,0.7,gbm_grid_model_18,1.455853,1.206587,1.220109
6,,0.9,0.096,0.99,3.0,130.0,100.0,0.7,gbm_grid_model_3,1.456012,1.206653,1.220498
7,,0.9,0.096,0.99,3.0,200.0,140.0,0.7,gbm_grid_model_43,1.456128,1.206701,1.220234
8,,0.9,0.096,0.99,3.0,200.0,130.0,0.7,gbm_grid_model_27,1.456194,1.206729,1.220267
9,,0.9,0.085,0.99,3.0,130.0,140.0,0.7,gbm_grid_model_33,1.456299,1.206772,1.220498


In [137]:
gbm_rmse.sort_values(by='valid_rmse', ascending=True)['model_ids'].tolist()

['gbm_grid_model_36',
 'gbm_grid_model_20',
 'gbm_grid_model_39',
 'gbm_grid_model_23',
 'gbm_grid_model_34',
 'gbm_grid_model_4',
 'gbm_grid_model_35',
 'gbm_grid_model_7',
 'gbm_grid_model_45',
 'gbm_grid_model_29',
 'gbm_grid_model_18',
 'gbm_grid_model_40',
 'gbm_grid_model_19',
 'gbm_grid_model_8',
 'gbm_grid_model_43',
 'gbm_grid_model_24',
 'gbm_grid_model_27',
 'gbm_grid_model_37',
 'gbm_grid_model_32',
 'gbm_grid_model_21',
 'gbm_grid_model_48',
 'gbm_grid_model_16',
 'gbm_grid_model_13',
 'gbm_grid_model_33',
 'gbm_grid_model_3',
 'gbm_grid_model_46',
 'gbm_grid_model_5',
 'gbm_grid_model_11',
 'gbm_grid_model_2',
 'gbm_grid_model_30',
 'gbm_grid_model_17',
 'gbm_grid_model_14',
 'gbm_grid_model_22',
 'gbm_grid_model_6',
 'gbm_grid_model_15',
 'gbm_grid_model_38',
 'gbm_grid_model_1',
 'gbm_grid_model_31',
 'gbm_grid_model_47',
 'gbm_grid_model_41',
 'gbm_grid_model_42',
 'gbm_grid_model_44',
 'gbm_grid_model_26',
 'gbm_grid_model_28',
 'gbm_grid_model_25',
 'gbm_grid_model_1

##  AutoML

### 🟦 GLM, DRF

In [99]:
glmdrf1 = H2OAutoML(max_models=20, nfolds = 5, keep_cross_validation_predictions=True,  include_algos = ['GLM', "DRF", 'StackedEnsemble'], seed=1234)
%time glmdrf1.train(x=predictors, y=response_col,training_frame=hf_train, validation_frame=hf_valid)
glmdrf1_lb = glmdrf1.leaderboard
glmdrf1_lb.head(rows=glmdrf1_lb.nrows)

glmdrf1_lb = h2o.automl.get_leaderboard(glmdrf1, extra_columns = "ALL")
glmdrf1_lb

AutoML progress: |
11:49:08.806: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 7.51 s, sys: 600 ms, total: 8.11 s
Wall time: 8min 41s


model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_BestOfFamily_1_AutoML_23_20220807_114908,1.21701,1.48111,0.892713,0.482911,1.48111,6939,0.086863,StackedEnsemble
GLM_1_AutoML_23_20220807_114908,1.22105,1.49097,0.898868,0.486638,1.49097,867,0.000727,GLM
XRT_1_AutoML_23_20220807_114908,1.25059,1.56396,0.911825,0.491912,1.56396,50379,0.048602,DRF
DRF_1_AutoML_23_20220807_114908,1.25593,1.57736,0.915039,0.492056,1.57736,36950,0.044641,DRF


In [102]:
for i, id in enumerate(glmdrf1_lb.as_data_frame()['model_id']) :
    mn = id[:3]+str(i+1)
    mod = h2o.get_model(id)
    print(mn + ' : ' + str(mod.rmse(valid=True)))
    yhat = mod.predict(hf_test)
    output = test_dt[['yyyymmdd', 'area', 'sex']].copy()
    output['frequency'] = yhat.as_data_frame()['predict']
    output = output.sort_values(by=['yyyymmdd', 'sex', 'area'], ascending=True)
    output.to_csv('/content/drive/MyDrive/weather_con/output/best_model' + '/output_'+ mn +'.csv', encoding='CP949',  index = False)
    model_path = h2o.save_model(model=mod, path='/content/drive/MyDrive/weather_con/models/best_model', force=True)

Sta1 : 1.229619589729529
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
GLM2 : 1.2362907189364616
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
XRT3 : 1.2527157978659336
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
DRF4 : 1.240696568282671
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


### 🟦 GBM

In [64]:
gbm1 = H2OAutoML(max_models=20, nfolds = 5, keep_cross_validation_predictions=True,  include_algos = ["GBM", 'StackedEnsemble'], seed=1234)
%time gbm1.train(x=predictors, y=response_col, training_frame=hf_train, validation_frame=hf_valid)
gbm1_lb = gbm1.leaderboard
gbm1_lb.head(rows=gbm1_lb.nrows)

gbm1_lb = h2o.automl.get_leaderboard(gbm1, extra_columns = "ALL")
gbm1_lb

AutoML progress: |
08:39:11.0: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 8.6 s, sys: 643 ms, total: 9.24 s
Wall time: 10min 21s


model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_19_20220807_83910,1.20622,1.45496,0.882074,0.477127,1.45496,12908,0.054697,StackedEnsemble
GBM_grid_1_AutoML_19_20220807_83910_model_3,1.20632,1.45521,0.88412,0.477647,1.45521,2798,0.010167,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_8,1.20946,1.46279,0.885715,0.478425,1.46279,2655,0.010553,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_15,1.21035,1.46494,0.886243,0.478173,1.46494,4377,0.010195,GBM
GBM_5_AutoML_19_20220807_83910,1.21205,1.46906,0.8886,0.479011,1.46906,3457,0.009928,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_13,1.21223,1.4695,0.886968,0.478821,1.4695,3387,0.01627,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_2,1.21258,1.47034,0.887724,0.479065,1.47034,4526,0.012371,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_5,1.21338,1.4723,0.888801,0.480075,1.4723,2235,0.012966,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_10,1.21368,1.47303,0.888455,0.478877,1.47303,5581,0.008999,GBM
GBM_grid_1_AutoML_19_20220807_83910_model_6,1.214,1.47379,0.887423,0.479584,1.47379,3490,0.013929,GBM


In [101]:
for i, id in enumerate(gbm1_lb.as_data_frame()['model_id'][:5]) :
    mn = 'gbm'+str(i+1)
    mod = h2o.get_model(id)
    print(mn + ' : ' + str(mod.rmse(valid=True)))
    yhat = mod.predict(hf_test)
    output = test_dt[['yyyymmdd', 'area', 'sex']].copy()
    output['frequency'] = yhat.as_data_frame()['predict']
    output = output.sort_values(by=['yyyymmdd', 'sex', 'area'], ascending=True)
    output.to_csv('/content/drive/MyDrive/weather_con/output/best_model' + '/output_'+ mn +'.csv', encoding='CP949',  index = False)
    model_path = h2o.save_model(model=mod, path='/content/drive/MyDrive/weather_con/models/best_model', force=True)

gbm1 : 1.2228725932585214
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
gbm2 : 1.223963340550397
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm3 : 1.2277667392670766
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm4 : 1.225905734299182
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm5 : 1.2242644697093434
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


### 🟦 XGB

In [68]:
xgb1 = H2OAutoML(max_models=20, nfolds = 5, keep_cross_validation_predictions=True,  include_algos = ["XGBoost", 'StackedEnsemble'], seed=1234)
%time xgb1.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)
xgb1_lb = xgb1.leaderboard
xgb1_lb.head(rows=xgb1_lb.nrows)

xgb1_lb = h2o.automl.get_leaderboard(xgb1, extra_columns = "ALL")
xgb1_lb

AutoML progress: |
09:41:53.32: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 9.53 s, sys: 919 ms, total: 10.5 s
Wall time: 13min 48s


model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_21_20220807_94153,1.20753,1.45814,0.884447,0.47809,1.45814,11870,0.050126,StackedEnsemble
XGBoost_grid_1_AutoML_21_20220807_94153_model_2,1.20916,1.46206,0.886775,0.479359,1.46206,2655,0.001265,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_5,1.20932,1.46246,0.886936,0.478954,1.46246,4234,0.001254,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_14,1.21074,1.46589,0.88837,0.479359,1.46589,2720,0.001275,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_15,1.21303,1.47144,0.889613,0.479898,1.47144,3569,0.001169,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_8,1.21337,1.47226,0.889244,0.479682,1.47226,2668,0.001246,XGBoost
XGBoost_3_AutoML_21_20220807_94153,1.22114,1.49117,0.891406,0.48087,1.49117,3681,0.001908,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_6,1.23362,1.52181,0.901561,0.484751,1.52181,4432,0.00182,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_11,1.23381,1.52228,0.901249,0.484953,1.52228,5579,0.002088,XGBoost
XGBoost_grid_1_AutoML_21_20220807_94153_model_17,1.24768,1.55671,0.912286,0.489909,1.55671,5277,0.002624,XGBoost


In [78]:
for i, id in enumerate(xgb1_lb.as_data_frame()['model_id'][:5]) :
    mn = 'xgb'+str(i+1)
    mod = h2o.get_model(id)
    print(mn + ' : ' + str(mod.rmse(valid=True)))
    yhat = mod.predict(hf_test)
    output = test_dt[['yyyymmdd', 'area', 'sex']].copy()
    output['frequency'] = yhat.as_data_frame()['predict']
    output = output.sort_values(by=['yyyymmdd', 'sex', 'area'], ascending=True)
    output.to_csv('/content/drive/MyDrive/weather_con/output/best_model' + '/output_'+ mn +'.csv', encoding='CP949',  index = False)
    model_path = h2o.save_model(model=mod, path='/content/drive/MyDrive/weather_con/models/best_model', force=True)

xgb1 : 1.225629267740529
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
xgb2 : 1.2288796286520753
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgb3 : 1.22985950063779
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgb4 : 1.2273541204984586
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgb5 : 1.2328669159140497
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


### 🟦 ANN

In [70]:
ann = H2OAutoML(max_models=20, nfolds = 5, keep_cross_validation_predictions=True, include_algos = ["DeepLearning", 'StackedEnsemble'], seed=1234)
%time ann.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)
ann_lb = ann.leaderboard
ann_lb.head(rows=ann_lb.nrows)

ann_lb = h2o.automl.get_leaderboard(ann, extra_columns = "ALL")
ann_lb

AutoML progress: |
09:55:49.858: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 43 s, sys: 4.72 s, total: 47.7 s
Wall time: 1h 9min 47s


model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_22_20220807_95549,1.20696,1.45675,0.883299,0.478099,1.45675,13935,0.105337,StackedEnsemble
DeepLearning_grid_1_AutoML_22_20220807_95549_model_7,1.21152,1.46778,0.883532,0.47771,1.46778,56458,0.005146,DeepLearning
DeepLearning_grid_1_AutoML_22_20220807_95549_model_5,1.21314,1.4717,0.882287,nan,1.4717,47129,0.003742,DeepLearning
DeepLearning_grid_1_AutoML_22_20220807_95549_model_1,1.21382,1.47337,0.883046,0.477386,1.47337,45165,0.013409,DeepLearning
DeepLearning_grid_1_AutoML_22_20220807_95549_model_2,1.21557,1.47762,0.892997,0.483107,1.47762,51808,0.005224,DeepLearning
DeepLearning_grid_2_AutoML_22_20220807_95549_model_2,1.21595,1.47853,0.888643,0.480531,1.47853,51590,0.008344,DeepLearning
DeepLearning_grid_1_AutoML_22_20220807_95549_model_3,1.21845,1.48462,0.89382,nan,1.48462,49428,0.013368,DeepLearning
DeepLearning_grid_1_AutoML_22_20220807_95549_model_6,1.21895,1.48584,0.891567,nan,1.48584,36405,0.003746,DeepLearning
DeepLearning_grid_3_AutoML_22_20220807_95549_model_6,1.21976,1.48781,0.888124,0.482105,1.48781,44677,0.005032,DeepLearning
DeepLearning_1_AutoML_22_20220807_95549,1.22042,1.48942,0.892445,0.480499,1.48942,5364,0.003074,DeepLearning


In [79]:
for i, id in enumerate(ann_lb.as_data_frame()['model_id'][:5]) :
    mn = 'ANN'+str(i+1)
    mod = h2o.get_model(id)
    print(mn + ' : ' + str(mod.rmse(valid=True)))
    yhat = mod.predict(hf_test)
    output = test_dt[['yyyymmdd', 'area', 'sex']].copy()
    output['frequency'] = yhat.as_data_frame()['predict']
    output = output.sort_values(by=['yyyymmdd', 'sex', 'area'], ascending=True)
    output.to_csv('/content/drive/MyDrive/weather_con/output/best_model' + '/output_'+ mn +'.csv', encoding='CP949',  index = False)
    model_path = h2o.save_model(model=mod, path='/content/drive/MyDrive/weather_con/models/best_model', force=True)

ANN1 : 1.2238495099639617
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
ANN2 : 1.2307294836066711
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
ANN3 : 1.2273767248570355
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
ANN4 : 1.2434400249561606
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
ANN5 : 1.2279958074032908
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


### Assignment

In [75]:
glmdrf1_lb.as_data_frame()['model_id']

0    StackedEnsemble_BestOfFamily_1_AutoML_20_20220...
1                       GLM_1_AutoML_20_20220807_84947
2                       XRT_1_AutoML_20_20220807_84947
3                       DRF_1_AutoML_20_20220807_84947
Name: model_id, dtype: object

In [112]:
name = ['stacked3', 'glm', 'xrt', 'drf']
for i, mn in enumerate(xgb1_lb.as_data_frame()['model_id'][:4]) :
    globals()[name[i]] = h2o.get_model(mn)

In [74]:
gbm1_lb.as_data_frame()['model_id'][:5]

0    StackedEnsemble_AllModels_1_AutoML_19_20220807...
1          GBM_grid_1_AutoML_19_20220807_83910_model_3
2          GBM_grid_1_AutoML_19_20220807_83910_model_8
3         GBM_grid_1_AutoML_19_20220807_83910_model_15
4                       GBM_5_AutoML_19_20220807_83910
Name: model_id, dtype: object

In [110]:
name = ['stacked_gbm', 'gbm1', 'gbm2', 'gbm3', 'gbm4']
for i, mn in enumerate(gbm1_lb.as_data_frame()['model_id'][:5]) :
    globals()[name[i]] = h2o.get_model(mn)

In [73]:
xgb1_lb.as_data_frame()['model_id'][:5]

0    StackedEnsemble_AllModels_1_AutoML_21_20220807...
1      XGBoost_grid_1_AutoML_21_20220807_94153_model_2
2      XGBoost_grid_1_AutoML_21_20220807_94153_model_5
3     XGBoost_grid_1_AutoML_21_20220807_94153_model_14
4     XGBoost_grid_1_AutoML_21_20220807_94153_model_15
Name: model_id, dtype: object

In [105]:
name = ['stacked_xgb', 'xgb1', 'xgb2', 'xgb3', 'xgb4']
for i, mn in enumerate(xgb1_lb.as_data_frame()['model_id'][:5]) :
    globals()[name[i]] = h2o.get_model(mn)

In [72]:
ann_lb.as_data_frame()['model_id'][:5]

0    StackedEnsemble_AllModels_1_AutoML_22_20220807...
1    DeepLearning_grid_1_AutoML_22_20220807_95549_m...
2    DeepLearning_grid_1_AutoML_22_20220807_95549_m...
3    DeepLearning_grid_1_AutoML_22_20220807_95549_m...
4    DeepLearning_grid_1_AutoML_22_20220807_95549_m...
Name: model_id, dtype: object

In [91]:
name = ['stacked_ann', 'ann1', 'ann2', 'ann3', 'ann4']
for i, mn in enumerate(ann_lb.as_data_frame()['model_id'][:5]) :
    globals()[name[i]] = h2o.get_model(mn)

---
# **ENSEMBLE**
---

In [158]:
### only automl
modelsss = [glm, drf, gbm1, gbm2, gbm3, gbm4, xgb1, xgb2, xgb3, xgb4, ann1, ann2, ann3, ann4]
ensemble_ml = H2OStackedEnsembleEstimator(model_id="ensemble_ml", base_models=modelsss)
%time ensemble_ml.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 132 ms, sys: 5.74 ms, total: 138 ms
Wall time: 2.78 s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_ml

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1.4315491761675303
RMSE: 1.196473642069699
MAE: 0.8765331009559442
RMSLE: 0.47547646767705815
R^2: 0.48128176283407864
Mean Residual Deviance: 1.4315491761675303
Null degrees of freedom: 10060
Residual degrees of freedom: 10049
Null deviance: 27766.173955217797
Residual deviance: 14402.816261421523
AIC: 32187.337332812116

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 1.4917759938109214
RMSE: 1.2213828203355905
MAE: 0.8986593208523168
RMSLE: 0.4777215812592463
R^2: 0.4807523468894279
Mean Residual Deviance: 1.4917759938109214
Null degrees of freedom: 6255
Residual degrees of freedom: 6244
Nu

In [138]:
ensemble_ml = H2OStackedEnsembleEstimator(model_id="ensemble_ml", base_models=modelsss, metalearner_algorithm="deeplearning")
%time ensemble_ml.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 1.58 s, sys: 156 ms, total: 1.74 s
Wall time: 1min 24s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_ml

No model summary for this model

ModelMetricsRegression: stackedensemble
** Reported on train data. **

MSE: 1.4179143794430205
RMSE: 1.1907621002715112
MAE: 0.867146696647308
RMSLE: 0.46843350565984515
Mean Residual Deviance: 1.4179143794430205

ModelMetricsRegression: stackedensemble
** Reported on validation data. **

MSE: 1.4935276585350215
RMSE: 1.2220996925517253
MAE: 0.895731716307274
RMSLE: 0.4760651892521234
Mean Residual Deviance: 1.4935276585350215


In [159]:
modelsss = [glm, gbm1, xgb1,  ann1]
ensemble_ml = H2OStackedEnsembleEstimator(model_id="ensemble_ml", base_models=modelsss, metalearner_algorithm="deeplearning")
%time ensemble_ml.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 1.48 s, sys: 143 ms, total: 1.62 s
Wall time: 1min 17s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_ml

No model summary for this model

ModelMetricsRegression: stackedensemble
** Reported on train data. **

MSE: 1.4043204610374338
RMSE: 1.1850402782342184
MAE: 0.874439631262712
RMSLE: 0.47167399278724625
Mean Residual Deviance: 1.4043204610374338

ModelMetricsRegression: stackedensemble
** Reported on validation data. **

MSE: 1.49636676451075
RMSE: 1.2232607099513781
MAE: 0.8991864383524548
RMSLE: 0.4763809198221766
Mean Residual Deviance: 1.49636676451075


In [139]:
ensemble_gbm = H2OStackedEnsembleEstimator(model_id="ensemble_gbm", base_models=gbm_grid.model_ids[:10])
%time ensemble_gbm.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 222 ms, sys: 13 ms, total: 235 ms
Wall time: 6.36 s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_gbm

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1.4329348480622477
RMSE: 1.197052566958631
MAE: 0.8800209673095098
RMSLE: 0.4750121378948272
R^2: 0.47777804379933386
Mean Residual Deviance: 1.4329348480622477
Null degrees of freedom: 10095
Residual degrees of freedom: 10087
Null deviance: 27703.57774914233
Residual deviance: 14466.910226036453
AIC: 32302.987255967113

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 1.4868009265358935
RMSE: 1.219344465906125
MAE: 0.8968663414873893
RMSLE: 0.4761503834662003
R^2: 0.4824840358409481
Mean Residual Deviance: 1.4868009265358935
Null degrees of freedom: 6255
Residual degrees of freedom: 6247
Null d

In [140]:
ensemble_gbm = H2OStackedEnsembleEstimator(model_id="ensemble_gbm", base_models=gbm_grid.model_ids[:10], metalearner_algorithm="deeplearning")
%time ensemble_gbm.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 1.64 s, sys: 182 ms, total: 1.82 s
Wall time: 1min 23s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_gbm

No model summary for this model

ModelMetricsRegression: stackedensemble
** Reported on train data. **

MSE: 1.484512848551832
RMSE: 1.2184058636397939
MAE: 0.8889768382670514
RMSLE: 0.4753150134440605
Mean Residual Deviance: 1.484512848551832

ModelMetricsRegression: stackedensemble
** Reported on validation data. **

MSE: 1.486701773334482
RMSE: 1.2193038068235833
MAE: 0.897284686391531
RMSLE: 0.4748129892959207
Mean Residual Deviance: 1.486701773334482


In [141]:
ensemble_drf = H2OStackedEnsembleEstimator(model_id="ensemble_drf", base_models=drf_grid.model_ids)
%time ensemble_drf.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 166 ms, sys: 5.78 ms, total: 172 ms
Wall time: 3.89 s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_drf

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1.3550245817971833
RMSE: 1.1640552314203925
MAE: 0.8530526302047505
RMSLE: 0.4673568169192354
R^2: 0.5116609492234189
Mean Residual Deviance: 1.3550245817971833
Null degrees of freedom: 9866
Residual degrees of freedom: 9862
Null deviance: 27378.632587006043
Residual deviance: 13370.027548592807
AIC: 31011.12096517939

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 1.4959259359580985
RMSE: 1.223080510824246
MAE: 0.895950222934691
RMSLE: 0.47713750203015975
R^2: 0.4793078621079283
Mean Residual Deviance: 1.4959259359580985
Null degrees of freedom: 6255
Residual degrees of freedom: 6251
Null d

In [143]:
ensemble_drf = H2OStackedEnsembleEstimator(model_id="ensemble_drf", base_models=drf_grid.model_ids,metalearner_algorithm="deeplearning")
%time ensemble_drf.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 1.55 s, sys: 145 ms, total: 1.69 s
Wall time: 1min 21s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_drf

No model summary for this model

ModelMetricsRegression: stackedensemble
** Reported on train data. **

MSE: 1.357331331027559
RMSE: 1.165045634740356
MAE: 0.8680668941187889
RMSLE: 0.4767723894058095
Mean Residual Deviance: 1.357331331027559

ModelMetricsRegression: stackedensemble
** Reported on validation data. **

MSE: 1.4972909637144953
RMSE: 1.2236384121604287
MAE: 0.9160110310103532
RMSLE: 0.48930713517617763
Mean Residual Deviance: 1.4972909637144953


---
# 🟩 **최종 모델**
---

In [154]:
mlist = glmdrf1_lb.as_data_frame()['model_id'].tolist()[1:2] + gbm1_lb.as_data_frame()['model_id'][1:2].tolist() + xgb1_lb.as_data_frame()['model_id'][1:2].tolist() + ann_lb.as_data_frame()['model_id'][1:2].tolist()

In [155]:
mlml = mlist + drf_grid.model_ids + gbm_grid.model_ids[:10]

In [160]:
ensemble_ml = H2OStackedEnsembleEstimator(model_id="ensemble_ml", base_models=mlml, metalearner_algorithm="deeplearning")
%time ensemble_ml.train(x=predictors, y=response_col,  training_frame=hf_train, validation_frame=hf_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
CPU times: user 2.04 s, sys: 190 ms, total: 2.23 s
Wall time: 1min 47s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_ml

No model summary for this model

ModelMetricsRegression: stackedensemble
** Reported on train data. **

MSE: 1.4318031792975792
RMSE: 1.1965797839248242
MAE: 0.8855589023220212
RMSLE: 0.4817234495889127
Mean Residual Deviance: 1.4318031792975792

ModelMetricsRegression: stackedensemble
** Reported on validation data. **

MSE: 1.4863621716484239
RMSE: 1.2191645383820937
MAE: 0.9023374849990307
RMSLE: 0.48012562984123175
Mean Residual Deviance: 1.4863621716484239


In [161]:
yhat2 = ensemble_ml.predict(hf_test)
output = test_dt[['yyyymmdd', 'area', 'sex']].copy()
output['frequency'] = yhat2.as_data_frame()['predict']
output = output.sort_values(by=['yyyymmdd', 'sex', 'area'], ascending=True)
output.to_csv('/content/drive/MyDrive/weather_con/output/' + '/output'+ '22080803' +'.csv', encoding='CP949',  index = False)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
